# import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer

In [2]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emotion")

Found cached dataset tweet_eval (C:/Users/User/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 3257
})

In [4]:
dataset["test"]

Dataset({
    features: ['text', 'label'],
    num_rows: 1421
})

In [5]:
dataset["validation"]

Dataset({
    features: ['text', 'label'],
    num_rows: 374
})

In [6]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


In [7]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)


In [8]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Get the input_ids from the tokenized dataset for train, validation, and test sets
train_input_ids = [example['input_ids'] for example in tokenized_dataset['train']]
val_input_ids = [example['input_ids'] for example in tokenized_dataset['validation']]
test_input_ids = [example['input_ids'] for example in tokenized_dataset['test']]

# Convert input_ids to PyTorch tensors
train_input_ids = torch.tensor(train_input_ids, dtype=torch.long)
val_input_ids = torch.tensor(val_input_ids, dtype=torch.long)
test_input_ids = torch.tensor(test_input_ids, dtype=torch.long)

# Concatenate the input_ids tensors along the first dimension
input_ids = torch.cat((train_input_ids, val_input_ids, test_input_ids), dim=0)


Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emotion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-9d05afcd1051738d.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emotion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-eea42245fea12e4d.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emotion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-a217fece7ee692df.arrow


In [9]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Get the attention_masks from the tokenized dataset for train, validation, and test sets
train_attention_mask = [example['attention_mask'] for example in tokenized_dataset['train']]
val_attention_mask = [example['attention_mask'] for example in tokenized_dataset['validation']]
test_attention_mask = [example['attention_mask'] for example in tokenized_dataset['test']]

# Convert attention_mask to PyTorch tensors
train_attention_mask = torch.tensor(train_attention_mask, dtype=torch.long)
val_attention_mask = torch.tensor(val_attention_mask, dtype=torch.long)
test_attention_mask = torch.tensor(test_attention_mask, dtype=torch.long)

# Concatenate the input_ids tensors along the first dimension
attention_masks = torch.cat((train_attention_mask, val_attention_mask, test_attention_mask), dim=0)


Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emotion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-9d05afcd1051738d.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emotion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-eea42245fea12e4d.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emotion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-a217fece7ee692df.arrow


In [10]:
print(input_ids)

tensor([[  101,   789,   160,  ...,     0,     0,     0],
        [  101,  1422, 20116,  ...,     0,     0,     0],
        [  101,  1302,  1133,  ...,     0,     0,     0],
        ...,
        [  101,  9202,   170,  ...,     0,     0,     0],
        [  101,  1398,  1142,  ...,     0,     0,     0],
        [  101,  5718,  1129,  ...,     0,     0,     0]])


In [11]:
print(attention_masks)

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [12]:
# Split the dataset into train, validation, and test sets
train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]
test_dataset = tokenized_dataset["test"]

In [13]:
print(input_ids)

tensor([[  101,   789,   160,  ...,     0,     0,     0],
        [  101,  1422, 20116,  ...,     0,     0,     0],
        [  101,  1302,  1133,  ...,     0,     0,     0],
        ...,
        [  101,  9202,   170,  ...,     0,     0,     0],
        [  101,  1398,  1142,  ...,     0,     0,     0],
        [  101,  5718,  1129,  ...,     0,     0,     0]])


In [14]:
# Convert the datasets to PyTorch tensors
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

In [15]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [16]:
import torch.nn as nn
from transformers import BertModel

class EmotionDetectionModel(nn.Module):
    def __init__(self, num_classes=4):
        super(EmotionDetectionModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits


In [17]:
print(input_ids)

tensor([[  101,   789,   160,  ...,     0,     0,     0],
        [  101,  1422, 20116,  ...,     0,     0,     0],
        [  101,  1302,  1133,  ...,     0,     0,     0],
        ...,
        [  101,  9202,   170,  ...,     0,     0,     0],
        [  101,  1398,  1142,  ...,     0,     0,     0],
        [  101,  5718,  1129,  ...,     0,     0,     0]])


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [19]:
# instantiate your model
emotion_model = EmotionDetectionModel(num_classes=4).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
print(emotion_model)

EmotionDetectionModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [21]:
# define your loss function
criterion = nn.CrossEntropyLoss()

# define your optimizer
optimizer = torch.optim.Adam(emotion_model.parameters(), lr=0.001)

In [22]:
# set up your training, validation, and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [23]:
print(input_ids)

tensor([[  101,   789,   160,  ...,     0,     0,     0],
        [  101,  1422, 20116,  ...,     0,     0,     0],
        [  101,  1302,  1133,  ...,     0,     0,     0],
        ...,
        [  101,  9202,   170,  ...,     0,     0,     0],
        [  101,  1398,  1142,  ...,     0,     0,     0],
        [  101,  5718,  1129,  ...,     0,     0,     0]])


In [24]:
print(attention_masks)

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [25]:
for batch in train_loader:
    print(batch)


{'label': tensor([3, 1, 0, 0, 0, 0, 1, 0, 3, 1, 1, 0, 1, 0, 0, 3]), 'input_ids': tensor([[  101, 18896,  3690,  ...,     0,     0,     0],
        [  101, 10616,  1110,  ...,     0,     0,     0],
        [  101,  1220,  1328,  ...,     0,     0,     0],
        ...,
        [  101,   137,  4795,  ...,     0,     0,     0],
        [  101, 11907, 25685,  ...,     0,     0,     0],
        [  101, 20735,  3650,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'label': tensor([0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 1, 3, 2, 0, 0, 0]), 'input_ids': tensor([[ 101, 2119,  137,  ...,    0,    0,    0],
        [ 101, 2814,  106,  ...,    0,    0,    0],
        [ 101,  137, 4795,  ...,    0,    0,    0],
        ...,
        [ 101,  180, 1424,  ...,    0,    0,    0],
        [ 101,  1

In [26]:
from tqdm import tqdm
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = emotion_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # print running loss for each batch
        running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        avg_acc = correct_predictions / total_predictions
        tqdm.write(f'Train Loss: {avg_loss:.3f}, Train Acc: {avg_acc:.3f}', end='\r')
    tqdm.write(f'Epoch {epoch+1}, Train Loss: {avg_loss:.3f}, Train Acc: {avg_acc:.3f}')

    print(f"Epoch {epoch+1} finished")


Epoch 1/1:   0%|          | 1/204 [00:17<57:31, 17.00s/it]

Epoch 1/1:   1%|          | 2/204 [00:32<53:34, 15.91s/it]

Epoch 1/1:   1%|▏         | 3/204 [00:46<51:18, 15.32s/it]

Epoch 1/1:   2%|▏         | 4/204 [01:00<49:16, 14.78s/it]

Epoch 1/1:   2%|▏         | 5/204 [01:14<48:09, 14.52s/it]

...

Epoch 1/1:  99%|█████████▊| 201/204 [1:18:32<01:12, 24.19s/it]

Epoch 1/1:  99%|█████████▉| 202/204 [1:18:56<00:48, 24.09s/it]

Epoch 1/1: 100%|█████████▉| 203/204 [1:19:20<00:23, 23.99s/it]

Epoch 1/1: 100%|██████████| 204/204 [1:19:35<00:00, 23.41s/it]

Epoch 1, Train Loss: 1.385, Train Acc: 0.374
Epoch 1 finished


In [27]:
# Validation loop
with torch.no_grad():
    emotion_model.eval()  # Set the model to evaluation mode
    valid_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in val_loader:
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # forward
        outputs = emotion_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # calculate running loss
        valid_loss += loss.item()

    avg_loss = valid_loss / len(val_loader)
    avg_acc = correct_predictions / total_predictions
    print(f'Validation Loss: {avg_loss:.3f}, Validation Acc: {avg_acc:.3f}')

Validation Loss: 1.302, Validation Acc: 0.428


In [28]:

# Test loop
with torch.no_grad():
    emotion_model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in test_loader:
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # forward
        outputs = emotion_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # calculate running loss
        test_loss += loss.item()

    avg_loss = test_loss / len(test_loader)
    avg_acc = correct_predictions / total_predictions
    print(f'Test Loss: {avg_loss:.3f}, Test Acc: {avg_acc:.3f}')


Test Loss: 1.338, Test Acc: 0.393


In [29]:
# save the model
torch.save(emotion_model.state_dict(), 'Models/emotion_model.pth')
